# 6. Route Details

## Observations

- `eta` information is available from this API.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp bmtc.apis.route_details

In [ ]:
#| hide

from nbdev.showdoc import *

In [ ]:
#| export

import json
import datetime
import time
import tarfile

from tqdm.auto import tqdm

import requests
import pandas as pd
pd.set_option('display.max_columns', None)

from fastcore.all import Path

from traffic_data_bengaluru.utils import *
from traffic_data_bengaluru.bmtc.apis.routes import get_routes

In [ ]:
#| export

import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)
logger = logging.getLogger(__name__)

In [ ]:
#| hide
#| eval: false

data_directory = get_data_directory() / "bmtc"

In [ ]:
# | export

def fetch_route_details(route_id: int, sleep_duration: float = 0.1):
    """Fetch route details for a given route ID from the BMTC API."""
    time.sleep(sleep_duration)
    url = "https://bmtcmobileapi.karnataka.gov.in/WebAPI/SearchByRouteDetails_v4"

    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36",
        "deviceType": "WEB",
        'lan': 'en'
    }

    payload = json.dumps({"routeid": int(route_id)})
    try:
        response = requests.post(url, headers=headers, data=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        print("Response text:", getattr(e.response, "text", None))
        return None

In [ ]:
#| hide
#| eval: false

sample_route_id = 2116
sample_route_details = fetch_route_details(route_id = sample_route_id)
sample_route_details

In [ ]:
# | export

def extract_route_details(route_details):
    details = []
    directions = ['up', 'down']
    for direction in directions:
        stations = route_details[direction]['data']
        for station in stations:
            for vehicle in station['vehicleDetails']:
                detail = {**vehicle, **{k: v for k, v in station.items() if k != "vehicleDetails"}}
                detail['direction'] = direction
                details.append(detail)
    df_details = pd.DataFrame(details)
    return df_details

In [ ]:
#| hide
#| eval: false

df_route_details = extract_route_details(sample_route_details)
print(df_route_details.shape)
df_route_details.head(3)

In [ ]:
# | export

def extract_map_data(route_details):
    map_data = []
    directions = ['up', 'down']
    for direction in directions:
        vehicles = route_details[direction]['mapData']
        map_data += vehicles
    df_map_data = pd.DataFrame(map_data)
    return df_map_data

In [ ]:
#| hide
#| eval: false

df_map_data = extract_map_data(sample_route_details)
print(df_map_data.shape)
df_map_data.head(3)

In [ ]:
# | export
def task_fetch_route_details(data_directory: Path):
    folder_name = f'{str(int(datetime.datetime.now().timestamp()))}'
    directory = data_directory / 'raw' / 'route_details' / folder_name
    directory.mkdir(exist_ok=True, parents=True)

    logger.info("Fetching route details ...")
    df_routes = get_routes(data_directory)
    for index, row in tqdm(df_routes.iterrows(), total = df_routes.shape[0], desc = 'Fetching route details'):
        route_details = fetch_route_details(route_id = row['route_id']) 
        with open(directory / f"{row['route_id']}.json", "w") as f:
            json.dump(route_details, f, indent = 4)

    tar_path = directory.with_suffix('.tar.gz')
    with tarfile.open(tar_path, "w:gz") as tar:
        tar.add(directory, arcname=directory.name)
    logger.info(f"Compressed {directory} into {tar_path}")
    
    # TODO: Delete the raw json files after successfully creating the compressed file.

In [ ]:
#| hide
#| eval: false

task_fetch_route_details(data_directory)

In [ ]:
# | export
def get_route_details(data_directory: Path):
    dfs = []
    directory = get_latest_directory(data_directory / 'raw' / 'route_details')
    for file in tqdm(extract_files(directory), total = len(extract_files(directory))):
        dfs.append(extract_route_details(read_file(file)))
    df_route_details = pd.concat(dfs, ignore_index=True)
    df_route_details = df_route_details.drop_duplicates().reset_index(drop = True)
    return df_route_details

In [ ]:
#| hide
#| eval: false

df_route_details = get_route_details(data_directory)

print(df_route_details.shape)
df_route_details.head(3)

In [ ]:
# | export
def get_map_data(data_directory: Path):
    dfs = []
    files = extract_files(get_latest_file(data_directory / 'raw' / 'route_details'))
    for file in tqdm(files, total = len(files)):
        dfs.append(extract_map_data(read_file(file)))
    df_map_data = pd.concat(dfs, ignore_index=True)
    df_map_data = df_map_data.drop_duplicates().reset_index(drop = True)
    return df_map_data

In [ ]:
#| hide
#| eval: false

df_map_data = get_map_data(data_directory)

print(df_map_data.shape)
df_map_data.head(3)

In [ ]:
# | export
def get_vehicle_count(route_details):
    """Return total number of vehicles in route for both directions."""
    vehicles_count = 0
    directions = ['up', 'down']
    for direction in directions:
        if direction in route_details:
            vehicles_count += len(route_details[direction]['mapData'])
    return vehicles_count

In [ ]:
#| hide
#| eval: false
get_vehicle_count(sample_route_details)

In [ ]:
#| hide
#| eval: false

vehicle_counts = {}
directory = get_latest_directory(data_directory / 'raw' / 'route_details')
for file in tqdm(extract_files(directory), total = len(extract_files(directory))):
    vehicle_counts[int(extract_file_name(file))] = get_vehicle_count(read_file(file))

df_vehicle_counts = pd.DataFrame({
    'route_id': vehicle_counts.keys(),
    'vehicle_count': vehicle_counts.values()
}).sort_values(by = 'vehicle_count', ascending = False)
print(df_vehicle_counts.shape)
df_vehicle_counts.head(3)

In [ ]:
#| hide
#| eval: false

pd.merge(
    df_vehicle_counts, 
    get_routes(data_directory),
    on = 'route_id',
    how = 'left'
).head(20)['route_id'].tolist()

In [ ]:
# | export
def task_fetch_route_details_v2(data_directory: Path):
    """v2 of fetch route details, routes with the most vehicles."""
    logger.info("Fetching route details v2 ...")

    route_ids = [1066, 1100, 995, 1558, 1659, 1387, 1508, 1086, 1133, 1732, 1507, 1324, 1705, 1109, 1419, 8543, 1173, 2802, 1164, 2254]
    for route_id in tqdm(route_ids, desc = 'Fetching route details v2'):

        # Create a directory for every route.
        directory = data_directory / 'raw' / 'route_details_v2' / str(route_id)
        directory.mkdir(exist_ok=True, parents=True)

        route_details = fetch_route_details(route_id = route_id)
        filename = f'{str(int(datetime.datetime.now().timestamp()))}'
        filepath = directory / f'{filename}.json'
        with open(filepath, "w") as f:
            json.dump(route_details, f, indent = 4)

In [ ]:
#| hide
#| eval: false

task_fetch_route_details_v2(data_directory)

In [ ]:
#| hide

import nbdev; nbdev.nbdev_export()